# MIND News Recommendation System

## Architecture Overview:
1. **News Encoder:** BERT/RoBERTa + Attention Pooling (Title → Vector)
2. **User Encoder:** Attention Pooling over history (History → User Vector)
3. **Prediction:** Dot product similarity + Softmax
4. **Training:** Negative Sampling with Cross-Entropy Loss

## Pipeline:
```
User History → News Encoder → User Encoder → User Vector
                                                  ↓
Candidate News → News Encoder → Candidate Vectors → Dot Product → Click Probability
```


## Import Libraries


In [1]:
%pip install --quiet numpy==1.26.4 pandas==2.3.3 matplotlib seaborn tqdm torch transformers scikit-learn sentence-transformers

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from src import *

# Set style for better plots
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

print("All libraries imported successfully!")


Note: you may need to restart the kernel to use updated packages.


/Users/lenhathoang/miniconda3/envs/recsys/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All libraries imported successfully!


## Configuration


In [2]:
print_config()

CONFIGURATION
NEWS_TRAIN_PATH      : /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/MIND/MINDsmall_train/news.tsv
NEWS_VAL_PATH        : /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/MIND/MINDsmall_dev/news.tsv
BEHAVIORS_TRAIN_PATH : /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/MIND/MINDsmall_train/behaviors.tsv
BEHAVIORS_VAL_PATH   : /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/MIND/MINDsmall_dev/behaviors.tsv
CHECKPOINT_PATH      : /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/checkpoints/mind_news_rec.pth
MODEL_NAME           : roberta-base
EMBEDDING_DIM        : 768
ATTENTION_QUERY_DIM  : 200
MAX_TITLE_LEN        : 10
MAX_HISTORY_LEN      : 20
NEG_SAMPLES          : 4
BATCH_SIZE           : 8
EPOCHS               : 3
LR                   : 0.0001
DEVICE               : mps
LOAD_CHECKPOINT      : False
DEBUG_SUBSET_SIZE    : 100


## Download dataset from Kaggle

In [3]:
download_dataset()

  4%|▍         | 54.0M/1.25G [00:07<02:53, 7.42MB/s]


KeyboardInterrupt: 

## News


### Load and Explore News Data


In [3]:
news_df = load_news_data()

print(f"\nNews Articles Samples:")
news_df.head(10)

Loading News Articles from /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/MIND/MINDsmall_train/news.tsv and /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/MIND/MINDsmall_dev/news.tsv...
  Training news: 51,282 articles
  Validation news: 42,416 articles
Loaded 65,238 unique news articles (combined)

News Articles Samples:


,news_id,title
0,N55528,"The Brands Queen Elizabeth, Prince Charles, an..."
1,N19639,50 Worst Habits For Belly Fat
2,N61837,The Cost of Trump's Aid Freeze in the Trenches...
3,N53526,I Was An NBA Wife. Here's How It Affected My M...
4,N38324,"How to Get Rid of Skin Tags, According to a De..."
5,N2073,Should NFL be able to fine players for critici...
6,N49186,It's been Orlando's hottest October ever so fa...
7,N59295,Chile: Three die in supermarket fire amid prot...
8,N24510,Best PS5 games: top PlayStation 5 titles to lo...
9,N39237,"How to report weather-related closings, delays"


### Tokenize News Articles


In [4]:
print(f"Initializing tokenizer: {config['MODEL_NAME']}...")
news_features = tokenize_news(news_df)
print(f"Tokenized {len(news_features):,} news articles (including <PAD>)")


Initializing tokenizer: roberta-base...


Tokenizing: 100%|██████████| 65238/65238 [00:06<00:00, 10439.16it/s]

Tokenized 65,239 news articles (including <PAD>)


## Create Training Samples


In [5]:
print("Creating training samples...")
print(f"Strategy: 1 positive + {config['NEG_SAMPLES']} negative samples per training example")

train_behaviors_df, val_behaviors_df = load_behaviors_data()

train_samples = create_behavior_samples(train_behaviors_df, 'train')
val_samples = create_behavior_samples(val_behaviors_df, 'val')

print(f"\nCreated {len(train_samples):,} training samples from {config['BEHAVIORS_TRAIN_PATH']}")
print(f"Created {len(val_samples):,} validation samples from {config['BEHAVIORS_VAL_PATH']}")

Creating training samples...
Strategy: 1 positive + 4 negative samples per training example
Training dataset path: /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/MIND/MINDsmall_train/behaviors.tsv
Validation dataset path: /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/MIND/MINDsmall_dev/behaviors.tsv
DEBUG MODE: Using 100 train and 100 val behaviors


Creating val samples: 100%|██████████| 100/100 [00:00<00:00, 59773.46it/s]


Created 137 training samples from /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/MIND/MINDsmall_train/behaviors.tsv
Created 100 validation samples from /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/MIND/MINDsmall_dev/behaviors.tsv


## Create PyTorch Dataset and DataLoader


In [6]:
class NewsDataset(Dataset):
    """PyTorch Dataset for news recommendation"""
    def __init__(self, samples, news_features, config):
        self.samples = samples
        self.news_features = news_features
        self.config = config
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        
        # Process history
        history_ids = sample['history']
        # Pad if shorter than max length
        if len(history_ids) < self.config['MAX_HISTORY_LEN']:
            history_ids = history_ids + ['<PAD>'] * (self.config['MAX_HISTORY_LEN'] - len(history_ids))
        
        # Stack history features
        hist_input_ids = torch.cat([
            self.news_features.get(nid, self.news_features['<PAD>'])['input_ids'] 
            for nid in history_ids
        ])
        hist_attn_mask = torch.cat([
            self.news_features.get(nid, self.news_features['<PAD>'])['attention_mask'] 
            for nid in history_ids
        ])
        
        # Process candidates
        candidate_ids = sample['candidates']
        cand_input_ids = torch.cat([
            self.news_features.get(nid, self.news_features['<PAD>'])['input_ids'] 
            for nid in candidate_ids
        ])
        cand_attn_mask = torch.cat([
            self.news_features.get(nid, self.news_features['<PAD>'])['attention_mask'] 
            for nid in candidate_ids
        ])
        
        return {
            'history_input_ids': hist_input_ids,
            'history_attn_mask': hist_attn_mask,
            'candidate_input_ids': cand_input_ids,
            'candidate_attn_mask': cand_attn_mask,
            'label': torch.tensor(sample['label'], dtype=torch.long)
        }

# Create datasets
print("Creating PyTorch datasets...")
train_dataset = NewsDataset(train_samples, news_features, config)
val_dataset = NewsDataset(val_samples, news_features, config)

print(f"Train dataset: {len(train_dataset):,} samples")
print(f"Validation dataset: {len(val_dataset):,} samples")

# Create dataloaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=config['BATCH_SIZE'], 
    shuffle=True,
    num_workers=0  # Set to 0 to avoid multiprocessing issues
)

val_loader = DataLoader(
    val_dataset,
    batch_size=1,  # Variable number of candidates per sample
    shuffle=False,
    num_workers=0
)

print(f"Train batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")


Creating PyTorch datasets...
Train dataset: 137 samples
Validation dataset: 100 samples
Train batches: 18
Validation batches: 100


## Initialize Model


In [7]:
model = NewsRecommender(config).to(config['DEVICE'])
optimizer = torch.optim.AdamW(model.parameters(), lr=config['LR'])
# optimizer = torch.optim.AdamW([
#     # Pretrained RoBERTa - lower learning rate
#     {'params': model.news_encoder.embedding.parameters(), 'lr': 1e-5},
#     # Custom attention layers - higher learning rate
#     {'params': model.news_encoder.attention_pooling.parameters(), 'lr': 1e-4},
#     {'params': model.user_attention.parameters(), 'lr': 1e-4}
# ], weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

print(f"Model initialized on {config['DEVICE']}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model initialized on mps
Total parameters: 124,953,632
Trainable parameters: 124,953,632


## Train the Model


In [8]:
train_model(model, train_loader, val_loader, optimizer, criterion, config['DEVICE'])

TRAINING FOR 3 EPOCHS


Epoch 1/3


Training: 100%|██████████| 18/18 [00:07<00:00,  2.54it/s, loss=2.6741]


Training Loss: 7.7125


Evaluating: 100%|██████████| 100/100 [00:03<00:00, 25.11it/s]


Validation Metrics:
  AUC: 0.5131
  MRR@5: 0.2458
  MRR@10: 0.2842
  NDCG@5: 0.2610
  NDCG@10: 0.3531
Model saved to /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/checkpoints/mind_news_rec.pth


Epoch 2/3


...

Validation Metrics:
  AUC: 0.5488
  MRR@5: 0.2517
  MRR@10: 0.2789
  NDCG@5: 0.2640
  NDCG@10: 0.3385
Model saved to /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/checkpoints/mind_news_rec.pth


Epoch 3/3


Training: 100%|██████████| 18/18 [00:06<00:00,  2.84it/s, loss=1.0546]


Training Loss: 1.6075


Evaluating: 100%|██████████| 100/100 [00:03<00:00, 30.38it/s]


Validation Metrics:
  AUC: 0.5312
  MRR@5: 0.2172
  MRR@10: 0.2336
  NDCG@5: 0.2525
  NDCG@10: 0.3040
Model saved to /Users/lenhathoang/Developer/HUST Projects/mind-news-recommendation/checkpoints/mind_news_rec.pth

TRAINING COMPLETE!
Final Training Loss: 1.6075
